<a href="https://colab.research.google.com/github/raymond92928/S5BAI/blob/main/Pytorch/Data_Processing_%E6%95%B8%E6%93%9A%E8%99%95%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handle outliers 處理異常值

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
scores={
    'Python': [90, 50, 70, 300, 80, 60, 62, 55, 76, 88, 90, 50, 70, 30, 80, 60, 62, 55, 76, 88],
    'Java': [300, 70, 90, 50, 40, 60, 77, 66, 50, 89, 30, 70, 90, 50, 40, 60, 77, 66, 50, 89],
    'PHP': [33, 220, 75, 85, 82, 90, 95, 56, 68, 65, 33, 22, 75, 85, 82, 90, 95, 56, 68, 65]
}

df =pd.DataFrame(scores)
print(df.shape)

(20, 3)


In [ ]:
outliers = {}
for i in range(df.shape[1]):
    min_t = df[df.columns[i]].mean() - (3*df[df.columns[i]].std()) # minimum threshold
    max_t = df[df.columns[i]].mean() + (3*df[df.columns[i]].std()) # maximum threshold
    
    count = 0
    
    for j in range(df.shape[0]):
        data = df.iloc[j, i]
        if data < min_t or data > max_t:
            count += 1
            
    percentage = count / df.shape[0]
    
    outliers[df.columns[i]] = "%.3f" %percentage
    
print(outliers)

{'Python': '0.050', 'Java': '0.050', 'PHP': '0.050'}


In [ ]:
for i in range(df.shape[1]):
    min_t = df[df.columns[i]].mean() - (3*df[df.columns[i]].std())
    max_t = df[df.columns[i]].mean() + (3*df[df.columns[i]].std())

    for j in range(df.shape[0]):
        data = df.iloc[j, i]
        if data < min_t or data > max_t:
            df.iloc[j, i] = np.NaN # change outliers to NaN
    
print(df)

    Python  Java   PHP
0     90.0   NaN  33.0
1     50.0  70.0   NaN
2     70.0  90.0  75.0
3      NaN  50.0  85.0
4     80.0  40.0  82.0
5     60.0  60.0  90.0
6     62.0  77.0  95.0
7     55.0  66.0  56.0
8     76.0  50.0  68.0
9     88.0  89.0  65.0
10    90.0  30.0  33.0
11    50.0  70.0  22.0
12    70.0  90.0  75.0
13    30.0  50.0  85.0
14    80.0  40.0  82.0
15    60.0  60.0  90.0
16    62.0  77.0  95.0
17    55.0  66.0  56.0
18    76.0  50.0  68.0
19    88.0  89.0  65.0


# Handle missing values 處理缺失值

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
scores={
    'Python': [90, 50, 70, np.NaN, 80, 60, np.NaN, 55, 76, 88],
    'Java': [np.NaN, 70, 90, 50, 40, np.NaN, 77, 66, np.NaN, 89],
    'PHP': [33, np.NaN, 75, 85, 82, 90, 95, 56, 68, np.NaN]
}


df =pd.DataFrame(scores)
print(df.shape)

(10, 3)


In [ ]:
print(df.isnull().sum())

Python    2
Java      3
PHP       2
dtype: int64


In [ ]:
print(df.isnull().sum().sum())

7


In [ ]:
df_orig = df.copy()
df2 = df.dropna() # drop the row/column include NaN
print(df2)

   Python  Java   PHP
2    70.0  90.0  75.0
4    80.0  40.0  82.0
7    55.0  66.0  56.0


In [ ]:
df3 = df.fillna(method = 'ffill') # propagate last valid observation forward to next valid
print(df3)

   Python  Java   PHP
0    90.0   NaN  33.0
1    50.0  70.0  33.0
2    70.0  90.0  75.0
3    70.0  50.0  85.0
4    80.0  40.0  82.0
5    60.0  40.0  90.0
6    60.0  77.0  95.0
7    55.0  66.0  56.0
8    76.0  66.0  68.0
9    88.0  89.0  68.0


In [ ]:
df4 = df.fillna(method = 'bfill') # use next valid observation to fill gap
print(df4)

   Python  Java   PHP
0    90.0  70.0  33.0
1    50.0  70.0  75.0
2    70.0  90.0  75.0
3    80.0  50.0  85.0
4    80.0  40.0  82.0
5    60.0  77.0  90.0
6    55.0  77.0  95.0
7    55.0  66.0  56.0
8    76.0  89.0  68.0
9    88.0  89.0   NaN


In [ ]:
python_avg = df.Python.mean().round().astype(int)
java_avg = df.Java.mean().round().astype(int)
php_avg = df.PHP.mean().round().astype(int)

print(python_avg, java_avg, php_avg)

71 69 73


In [ ]:
df.Python.fillna(value = python_avg, inplace = True)
df.Java.fillna(value = java_avg, inplace = True)
df.PHP.fillna(value = php_avg, inplace = True)

print(df)

   Python  Java   PHP
0    90.0  69.0  33.0
1    50.0  70.0  73.0
2    70.0  90.0  75.0
3    71.0  50.0  85.0
4    80.0  40.0  82.0
5    60.0  69.0  90.0
6    71.0  77.0  95.0
7    55.0  66.0  56.0
8    76.0  69.0  68.0
9    88.0  89.0  73.0


# 數據縮放

## Normalizatoin 正規化

$$
x_{norm} = \frac{x_i-\min{(x)}}{\max{(x)}-\min{(x)}}
$$

In [ ]:
df_orig = df
df = (df - df.min())/(df.max() - df.min())
print(df)

   Python  Java       PHP
0   1.000  0.58  0.000000
1   0.000  0.60  0.645161
2   0.500  1.00  0.677419
3   0.525  0.20  0.838710
4   0.750  0.00  0.790323
5   0.250  0.58  0.919355
6   0.525  0.74  1.000000
7   0.125  0.52  0.370968
8   0.650  0.58  0.564516
9   0.950  0.98  0.645161


## Starnardization 標準化

$$
x_{stand} = \frac{x_i - \text{mean}(x)}{\text{std}(x)}
$$

In [ ]:
df = df_orig
df = (df-df.mean())/(df.std())
print(df)

     Python      Java       PHP
0  1.428751  0.006520 -2.222222
1 -1.595061  0.071724  0.000000
2 -0.083155  1.375794  0.111111
3 -0.007560 -1.232347  0.666667
4  0.672798 -1.884382  0.500000
5 -0.839108  0.006520  0.944444
6 -0.007560  0.528149  1.222222
7 -1.217084 -0.189090 -0.944444
8  0.370417  0.006520 -0.277778
9  1.277560  1.310591  0.000000


# split dataset折分數據集
In particular, three data sets are commonly used in different stages of the creation of the model: training (訓練集), validation (驗證集), and test sets (測試集).

## split by index

In [ ]:
df = df_orig
x_shuffle = df.sample(frac = 1, random_state = 0) # frac = 1 mean used all data
print(x_shuffle)

   Python  Java   PHP
2    70.0  90.0  75.0
8    76.0  69.0  68.0
4    80.0  40.0  82.0
9    88.0  89.0  73.0
1    50.0  70.0  73.0
6    71.0  77.0  95.0
7    55.0  66.0  56.0
3    71.0  50.0  85.0
0    90.0  69.0  33.0
5    60.0  69.0  90.0


In [ ]:
train_end = int(len(x_shuffle) * 0.6)
dev_end = int(len(x_shuffle) * 0.8)
print(train_end, dev_end)

6 8


In [ ]:
x_train = x_shuffle.iloc[:train_end, :]
x_dev = x_shuffle.iloc[train_end:dev_end, :]
x_test = x_shuffle.iloc[dev_end:, :]
print(x_train.shape, x_dev.shape, x_test.shape)

(6, 3) (2, 3) (2, 3)


In [ ]:
print(x_dev)

   Python  Java   PHP
7    55.0  66.0  56.0
3    71.0  50.0  85.0


## train_test_split()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_new, x_test2 = train_test_split(df, test_size = 0.2, random_state = 0)
print(x_new.shape, x_test2.shape)

(8, 3) (2, 3)


In [ ]:
x_train2, x_dev2 = train_test_split(x_new, test_size = 0.25, random_state = 0)
print(x_train2)

   Python  Java   PHP
9    88.0  89.0  73.0
5    60.0  69.0  90.0
6    71.0  77.0  95.0
4    80.0  40.0  82.0
3    71.0  50.0  85.0
7    55.0  66.0  56.0


# Exercises
Predict the release year of a song

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

torch.manual_seed(0)

In [ ]:
data=pd.read_csv('data/YearPredictionMSD.csv',nrows=50000,header=None)
print(data.head())
print(data.shape)

     0         1         2         3         4         5         6         7   \
0  2001  49.94357  21.47114  73.07750   8.74861 -17.40628 -13.09905 -25.01202   
1  2001  48.73215  18.42930  70.32679  12.94636 -10.32437 -24.83777   8.76630   
2  2001  50.95714  31.85602  55.81851  13.41693  -6.57898 -18.54940  -3.27872   
3  2001  48.24750  -1.89837  36.29772   2.58776   0.97170 -26.21683   5.05097   
4  2001  50.97020  42.20998  67.09964   8.46791 -15.85279 -16.81409 -12.48207   

         8         9   ...        81         82        83        84        85  \
0 -12.23257   7.83089  ...  13.01620  -54.40548  58.99367  15.37344   1.11144   
1  -0.92019  18.76548  ...   5.66812  -19.68073  33.04964  42.87836  -9.90378   
2  -2.35035  16.07017  ...   3.03800   26.05866 -50.92779  10.93792  -0.07568   
3 -10.34124   3.55005  ...  34.57337 -171.70734 -16.96705 -46.67617 -12.51516   
4  -9.37636  12.63699  ...   9.92661  -55.95724  64.92712 -17.72522  -1.49237   

         86         87    

In [ ]:
cols=data.columns # numbers of columns of data
num_cols=data._get_numeric_data().columns # numbers of columns of numeric data
print(list(set(cols)-set(num_cols)))

[]


In [ ]:
print(data.isnull().sum().sum()) # check NaN

0


In [ ]:
outliers=[]
for i in range(data.shape[1]):    
    min_t=data[data.columns[i]].mean()-(3*data[data.columns[i]].std())
    max_t=data[data.columns[i]].mean()+(3*data[data.columns[i]].std())
    count=0
    for j in data[data.columns[i]]:
        if j<min_t or j>max_t:
            count += 1
    percentage=count/data.shape[0]
    #print(i, percentage)
    if percentage > 0.05:
        outliers.append(i)    

print(outliers)

[]


In [ ]:
x=data.iloc[:,1:]
y=data.iloc[:,0]
x=(x-x.mean())/x.std() # Starnardization
print(x.head())

         1         2         3         4         5         6         7   \
0  1.082657  0.382437  1.841985  0.459652 -0.480074 -0.282606 -1.590785   
1  0.880874  0.321953  1.763666  0.717085 -0.165507 -1.188896  0.777905   
2  1.251484  0.588929  1.350579  0.745944  0.000857 -0.703401 -0.066747   
3  0.800148 -0.082240  0.794774  0.081829  0.336246 -1.295366  0.517369   
4  1.253660  0.794806  1.671781  0.442438 -0.411071 -0.569426 -0.712128   

         8         9         10  ...        81        82        83        84  \
0 -1.300854  0.378336 -0.683719  ... -0.086005  0.099339  0.148291 -0.255625   
1  0.122576  1.420531  0.401198  ... -0.316635  0.301448 -0.063611  0.031855   
2 -0.057380  1.163637 -0.090081  ... -0.399185  0.567666 -0.749508 -0.301984   
3 -1.062869 -0.029679 -1.282306  ...  0.590596 -0.583396 -0.472129 -0.904164   
4 -0.941459  0.836414 -0.160630  ... -0.182976  0.090307  0.196753 -0.601570   

         85        86        87        88        89        90  
0  0

In [ ]:
# split to two sets，x_new : x_test = 80:20
x_new, x_test, y_new, y_test=train_test_split(x, y, test_size=0.2, random_state=0)

# split to two sets again, x_train : x_dev = 75:25
x_train, x_dev, y_train, y_dev=train_test_split(x_new, y_new, test_size=0.25, random_state=0)

print(x_train.shape, x_dev.shape, x_test.shape)

(30000, 90) (10000, 90) (10000, 90)


In [ ]:
x_train_torch=torch.tensor(x_train.values).float()
y_train_torch=torch.tensor(y_train.values).float().unsqueeze(1)
x_dev_torch=torch.tensor(x_dev.values).float()
y_dev_torch=torch.tensor(y_dev.values).float().unsqueeze(1)
x_test_torch=torch.tensor(x_test.values).float()
y_test_torch=torch.tensor(y_test.values).float().unsqueeze(1)

print(x_train_torch.shape, y_train_torch.shape)


torch.Size([30000, 90]) torch.Size([30000, 1])


In [ ]:
model=nn.Sequential(
    nn.Linear(x_train.shape[1],200),
    nn.ReLU(),
    nn.Linear(200,50),
    nn.ReLU(),    
    nn.Linear(50,1)
)

In [ ]:
device="cuda"
model=model.to(device)
x_train_torch=x_train_torch.to(device)
y_train_torch=y_train_torch.to(device)
x_dev_torch=x_dev_torch.to(device)
y_dev_torch=y_dev_torch.to(device)

In [ ]:
myloss=torch.nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [ ]:
epochs=5001
for epoch in range(epochs):
    model.train()
    y_pred=model(x_train_torch)      
    train_loss=myloss(y_pred,y_train_torch)
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()   
    
    if (epoch%400==0):        
        with torch.no_grad():
            model.eval()
            y_pred2=model(x_dev_torch)            
            valid_loss=myloss(y_pred2,y_dev_torch)
            
        print(f"epoch={epoch},  train_loss:{train_loss.item():.3f},valid_loss:{valid_loss.item():.3f}")
        
        if train_loss.item() < 81:
            break


epoch=0,  train_loss:3994187.250,valid_loss:3991035.500
epoch=400,  train_loss:8711.881,valid_loss:12555.356
epoch=800,  train_loss:279.475,valid_loss:782.625
epoch=1200,  train_loss:113.952,valid_loss:384.486
epoch=1600,  train_loss:90.559,valid_loss:293.810
epoch=2000,  train_loss:83.806,valid_loss:259.099
epoch=2400,  train_loss:94.531,valid_loss:247.058
epoch=2800,  train_loss:84.640,valid_loss:238.893
epoch=3200,  train_loss:90.362,valid_loss:266.793
epoch=3600,  train_loss:96.531,valid_loss:232.401
epoch=4000,  train_loss:80.850,valid_loss:236.639


In [ ]:
model=model.to("cpu")
pred=model(x_test_torch)
test_loss=myloss(pred,y_test_torch)
print(f"test_loss: {test_loss.item():.3f}")

test_loss: 356.272


In [ ]:
for i in range(100,110):
    print(f"truth:{y_test_torch[i].item():.0f}, pred:{pred[i].item():.0f}")

truth:2006, pred:2011
truth:2003, pred:1998
truth:2002, pred:2001
truth:2005, pred:2100
truth:2007, pred:2001
truth:2008, pred:1999
truth:2001, pred:2004
truth:2006, pred:2004
truth:2008, pred:2006
truth:1966, pred:1988
